<a href="https://colab.research.google.com/github/denizMoC/MetabolismofCities/blob/main/S_Sankey_Bod%C3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries 

In [ ]:
import plotly.graph_objects as go

from google.colab import data_table
data_table.enable_dataframe_formatter()

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()


gc = gspread.authorize(creds)

## Data

In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1pjlV7S3BudM7GWxlw0NOBUzu1QUGf-ZpzVzivNPJANA/edit?usp=sharing').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows[1:])

#Adding the headers
df.set_axis(["FROM", "TO", "value", "flow_label", "flow_color","node_color"],axis=1,inplace=True)
df

[['From', 'To', 'Value', 'flow_label', 'flow_color', 'node_color'], ['Imports', 'Exports', '139.17', 'Imports --> Exports', 'rgba(0, 77, 118, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Domestic Extration', 'Exports', '8.13', 'Domestic Extration --> Exports', 'rgba(0, 77, 118, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Imports', 'Processed materials', '291.82', 'Imports --> Processed materials', 'rgba(0, 77, 118, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Domestic Extration', 'Processed materials', '47.11', 'Domestic Extration --> Processed materials', 'rgba(0, 77, 118, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Processed materials', 'Energetic use', '93.63', 'Processed materials --> Energetic use', 'rgba(0, 77, 118, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Processed materials', 'Material use', '265.48', 'Processed materials --> Material use', 'rgba(0, 77, 118, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Energetic use', 'DPO emissions', '55.28', 'Energetic use --> DPO emissions', 'rgba(0, 77, 118, 0.5)', 'rgba(239,

,FROM,TO,value,flow_label,flow_color,node_color
0,Imports,Exports,139.17,Imports --> Exports,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"
1,Domestic Extration,Exports,8.13,Domestic Extration --> Exports,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"
2,Imports,Processed materials,291.82,Imports --> Processed materials,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"
3,Domestic Extration,Processed materials,47.11,Domestic Extration --> Processed materials,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"
4,Processed materials,Energetic use,93.63,Processed materials --> Energetic use,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"
5,Processed materials,Material use,265.48,Processed materials --> Material use,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"
6,Energetic use,DPO emissions,55.28,Energetic use --> DPO emissions,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"
7,Energetic use,EoL Waste,38.35,Energetic use --> EoL Waste,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"
8,Material use,EoL Waste,54.20,Material use --> EoL Waste,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"
9,Material use,In-use stocks,211.28,Material use --> In-use stocks,"rgba(0, 77, 118, 0.5)","rgba(239, 239, 239, 0.5)"


In [ ]:
# We create the 3 list we need
source = df['FROM'].tolist()
target = df['TO'].tolist()
flow_color = df['flow_color'].tolist()
node_color = df['node_color'].tolist()
value = df['value'].tolist()
flow_label = df['flow_label'].tolist()
node = pd.unique(source + target).tolist()

# Function to create a list of unique nodes and assign them 0 based number
def assign_number(node, source, target):
  """
  It takes in a list of nodes, a list of source nodes, and a list of target nodes, and returns a list
  of source nodes and a list of target nodes, where each node is replaced by its index in the list of
  nodes
  
  :param node: the list of nodes in the graph
  :param source: the source node
  :param target: the target node
  :return: The source and target lists are being returned.
  """
  for i in range(len(node)):
      for j in range(len(source)):
          if node[i] == source[j]:
              source[j] = i
      for k in range(len(target)):
          if node[i] == target[k]:
              target[k] = i
  return source, target

source, target = assign_number(node, source, target) # assign number to source and target

print(node)
print(source)
print(target)
print(value)

['Imports', 'Domestic Extration', 'Processed materials', 'Energetic use', 'Material use', 'In-use stocks', 'EoL Waste', 'Exports', 'DPO emissions', 'DPO waste']
[0, 1, 0, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6]
[7, 7, 2, 2, 3, 4, 8, 6, 6, 5, 6, 7, 2, 9]
['139.17', '8.13', '291.82', '47.11', '93.63', '265.48', '55.28', '38.35', '54.20', '211.28', '0.94', '4.68', '20.18', '68.63']


In [ ]:
# Function to plot a sankey diagram

fig = go.Figure(data=[go.Sankey(
    node = dict(
        # You can set a whole lot of stylistic options, see https://plotly.com/python/reference/sankey/
        thickness = 10,
        line = dict(width = 0),
        color = node_color,
        # This is the list of nodes: the TO and FROM in your spreadsheet
        # Each node must be included once and only once
        label = node,
    ),
    # These are the connections between the nodes, the flows between them
    # They're made up of three parts: the source, the target, and the value
    # The numbers correspond to the labels (0-based)
    # So, the first item of each array is taken to form the first connection
    # The it continues with the second, third, and so forth until the end
    link = dict(
      source= source,
      target = target,
      color = flow_color,
      value = value,
      label = flow_label,
    
      # You can optionally add labels that will show up when hovering over the connection
      # For connections without a label, simply put quotation marks without any text: ""
      # labels = ["label for first connection", "", "", ...]
  ))])
# This updates some layout stuff so that it matches MOC styles
# See https://plotly.com/python/reference/sankey/ for options
fig.update_layout(
    hovermode = 'x',
    font=dict(size = 14, color = 'black', family = 'Lato'),
    #plot_bgcolor='rgba(255,255,255,0)',
    #paper_bgcolor='rgba(255,255,255,0)',
)
sankey = fig.to_html(full_html=True)
context = {
    "sankey": sankey,
}

fig.update_layout(title_text="", font_size=10)

#To change the arrangement
fig.update_traces(arrangement="freeform", 
                  node = {"label": node,
#[1'Imports', 2'Domestic Extration', 3'Processed materials', 4'Energetic use', 5'Material use', 6'In-use stocks', 7'EoL Waste', 8'Exports', 9'DPO emissions', 10'DPO waste']
                              "x": [0,    0, 0.25,0.5, 0.5, 0.7, 0.8,   1,   1,   1],
                              "y": [1, 0.01,  0.7,0.4, 0.7,  0.8, 0.6, 0.1, 0.5, 0.6],
                          'pad':100},  # 10 Pixels 
                  
node_pad=15, 
selector=dict(type='sankey'))

# Export the HTLM
fig.write_html("Sankey - Bodø")

#To display
fig.show()


In [ ]:
node


['Imports',
 'Domestic Extration',
 'Processed materials',
 'Energetic use',
 'Material use',
 'In-use stocks',
 'EoL Waste',
 'Exports',
 'DPO emissions',
 'DPO waste']